<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>

# Laboratory work #1. Introduction to FHIR resources

Meet John Thompson. He is a 36 year old (date of birth is *08-02-1982*) man from *Philadelphia*. He is thinking of visiting our clinical center because of some health problems. John calls our center and asks if he can visit us.

Let’s check if he has visited our center before.

## 1 Introduction

### 1.1 Import libraries

For the beginning we should import libraries `fhirpy` and `os`.  
Also we import `pprint` helper function from local `utils.py` file. We'll use `pprint` for display some structures.

In [ ]:
import os
import fhirpy

from utils import pprint

### 1.2 Create an instance of connection

To load data from FHIR server we should initiate `FHIRClient` class from `fhirpy` package.  
We pass `url` and `authorization` arguments from environment.

In [ ]:
client = fhirpy.FHIRClient(
    fhir_version='4.0.0',
    url=os.environ['BOX_URL'] + '/fhir', 
    authorization=os.environ['BOX_AUTHORIZATION'])

Now, we are able to operate with FHIR resources using `client`.

## 2 Load data from the FHIR server

### 2.1 Load list of patients

The `Patient` resource covers data about patients and animals involved in a wide range of health-related activities, including:

* Theurapeutic activities
* Psychiatric care
* Social services
* Pregnancy care
* Nursing and assisted living
* Dietary services
* Tracking of personal health and exercise data

The data in the Resource cover the "who" information about the patient: its attributes are focused on the demographic information necessary to support the administrative, financial and logistic procedures. A `Patient` record is generally created and maintained by each organization providing care for a patient.

Let's try to fetch all patients in the database using `resources` method.  
This method returns a 'lazy object' (an instance of `FHIRSearchSet`), which provides some helpful methods for building queries. The most important method which we are going to use is `fetch_all`. Using it, we can execute built queries and load all records suitable for our query.

In [ ]:
patients = client.resources('Patient').fetch_all()

We built a simple query without any filters and sortings. Executing this query we load all patient records which are represented as dict-like object (an instance of `FHIRResource`).

Let's try to display the result.

In [ ]:
patients

As we can see from the output above, we have a lot of patient resources with different IDs.  
To get more details, we can get some fields using `get` and `get_by_path` methods.  

`get(field_name)` method receives field name as the first argument.  
`get_by_path(path)` method receives path as string (`'name.0.given.0'`) or as a list (`['name', 0, 'given', 0]`)  

Let's try to iterate over `patients` list and display their ids and full name.

In [ ]:
for patient in patients:
    print('{0} {1} {2}'.format(
        patient.get('id'), 
        patient.get_by_path('name.0.family'),
        patient.get_by_path('name.0.given.0')))

### 2.2 Sorting results

Also, we can sort the result, for example, by name using `sort` method.  
Please, pay attention, that `sort` receives multiple parameters and all possible parameters described in the [official FHIR specification](https://www.hl7.org/fhir/patient.html#search).

In [ ]:
patients = client.resources('Patient').sort('name').fetch_all()

Now we can display the result and see what changed.

In [ ]:
for patient in patients:
    print('{0} {1} {2}'.format(
        patient.get('id'), 
        patient.get_by_path('name.0.family'),
        patient.get_by_path('name.0.given.0')))

As we can see, the list is very long and it may be too difficult to find the particular patient especially if we have thousands of entries.  
To minimize the result, FHIR API provides special search tools.  

### 2.3 Search through patients' resources

The `Patient` resource has many search parameters. You can read more about them in the [official FHIR specification](https://www.hl7.org/fhir/patient.html#search).  



For searching we should use `search` method on a search set. If we want to find, for example, all patients with the first name `John` and the last name `Thompson` we should use intersection search, passing list of values, for example, `search(name=['John', 'Thompson'])`. This is known as an AND search parameter.  
If we wanted to find all patients with name `John` or `Carl`, we would use `search(name='John,Carl')`. This is known as an OR search parameter.


Let's try to search for a patient by a parameter `name`.  
This param is used for searching by string fields in the patient's name, including family, given, prefix, suffix, and/or text.  

In [ ]:
patients = client.resources('Patient').search(name=['John', 'Thompson']).fetch_all()
patients

We found nothing. Maybe his record has a typo in the name. Now let's try to search by other parameters, for example, 
by birth date. The search parameter has name `birthdate` according to the [FHIR specification](https://www.hl7.org/fhir/patient.html#search).  

In [ ]:
patients = client.resources('Patient').search(birthdate='1982-08-02').fetch_all()
patients

Also, we can try to find all patients who were born in `Philadelphia`. For this purpose we should use `address` search param.

In [ ]:
patients = client.resources('Patient').search(address='Philadelphia').fetch_all()
patients

So, we didn’t find our client in the system, so, we should create a record for him. But before we do that, let's learn more about how different fields should be stored.

## 3 Patient resource structure

Let's try to load one patient resource and understand this structure.  
We don't need to load all patient's records because they have a similar structure, so we can load only one record using `first` method on the search set.  
This method returns only one resource representation (an instance of `FHIRResource`).

In [ ]:
patient = client.resources('Patient').first()

As you know, FHIRResource is a dict-like object. It means, we can display some fields from the resource using an access by key.

In [ ]:
patient['id']

Let's see how the patient name looks.  

### 3.1 Name

The name is represented using `HumanName` type. 

Names might be changed and people may have different names in different contexts. So, a patient name is always represented as a list even if the patient has only one name. 

The most popular fields in this structure are:
* given - given names, not only first name (e.g. John). It should be a list of names, even if the person has only one.
* family - family name (e.g. Tompson)
* use - in which context this name applies
* text - text representation of the full name (e.g. John Tompson)


More information you can find more information in the [FHIR specification](https://www.hl7.org/fhir/datatypes.html#humanname). 

In [ ]:
pprint(patient['name'])

### 3.2 Address

The address type may be used to convey addresses for use in delivering mail as well as for visiting locations which might not be valid for mail delivery. There are a variety of address formats defined around the world.

More information you can find in the [FHIR specification](https://www.hl7.org/fhir/datatypes.html#address). 

In [ ]:
pprint(patient['address'])

### 3.3 Contact information

Contact information is stored in `telecom` field. This fields contains details for all kinds of technology-mediated contact points for a person, including telephone, email, etc.

In [ ]:
pprint(patient['telecom'])

## 4 Editing patient resource

The `Patient` resource can include different fields, which are described in the [official FHIR specification](https://www.hl7.org/fhir/patient.html#resource). 
Now we are familiar with some of them: `name`, `address` and `telecom`. Let's create a new record for the patient.

### 4.1 Create a new patient record

We can create a new instance of resource using `resource` method. This method creates a new resource representation in the memory.

In [ ]:
patient = client.resource('Patient')
patient

This line doesn't save the patient to the database. So, we can change any attributes of resource before it will be really saved.

In [ ]:
patient['name'] = [
    {
        'given': ['John'],
        'family': 'Thompson',
        'use': 'official',
        'prefix': ['Mr.'],
    }
]

Also we can specify date of birth, using `birthDate` field. We should pass a date in format `year-month-day`. Let's do it by yourself.

In [ ]:
patient['birthDate'] = ''  # Write code here

To save the resource, we should use `save` method.

In [ ]:
patient.save()

This line doesn't return anything in case of success but mutates the patient resource. Now, the `patient` has an additional meta information and unique ID.  
We can look at this information using attributes `id` and `meta`.

In [ ]:
patient['id']

In [ ]:
patient['meta']

### 4.2 Edit a patient record

Finally, we should update the patient’s details.  
Let's edit the  address - the new one is `1818 Market St,apartment 100, Philadelphia, PA 19103`

You need to fill in the address by yourself using fields from [FHIR specification](https://www.hl7.org/fhir/datatypes.html#address). 


In [ ]:
patient['address'] = [
    {
        # Write code here
    }
]

And set the new telephone number: `(215) 352-3801`.

In [ ]:
patient['telecom'] = [
    {
        # Write code here
    }
]

And finally, save the changes to the database.

In [ ]:
patient.save()

## 5 Check laboratory results

Now, we need to make sure that we have our patient record in the database.  

Let's try to search for the patient again. For this operation we should use the same query as we did it before.

In [ ]:
patients = client.resources('Patient').search(name=['John', 'Thompson']).fetch_all()
pprint(patients)

If you followed the instructions, you should have the following fields in the output:

* id
* resourceType
* birthDate
* name
* telecom
* address

## 6 Summary

From this laboratory work we learned how to:
* load patients from the FHIR server
* search by different fields
* sort the results
* display different resorce's attributes
* create patient record
* edit patient record



<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>